# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!

## Please note:

When you run the pip installs in the first cell below, you might get this error - it can be safely ignored - it sounds quite severe, but it doesn't seem to affect anything else in this project!


> ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.



In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2 gradio

In [ ]:
# imports

import os
import requests
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr

In [ ]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [ ]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Initialize Llama model and tokenizer

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto",
    quantization_config=quant_config
)

In [ ]:
# Generate meeting minutes

def generate_minutes(transcription, model, tokenizer, progress=gr.Progress()):
    progress(0.6, desc="Generating meeting minutes from transcript...")

    system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
    user_prompt = f"Below is an extract transcript of a meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0])

    # Clean up the response, keep only the minutes
    progress(0.9, desc="Cleaning and formatting minutes...")
    response = response.split("<|end_header_id|>")[-1].strip().replace("<|eot_id|>","")

    return response

In [ ]:
# Transcribe the uploaded audio file using OpenAI's Whisper model

def transcribe_audio(audio_path, progress=gr.Progress()):
    progress(0.3, desc="Creating transcript from audio...")

    try:
        with open(audio_path, "rb") as audio_file:
            transcription = openai.audio.transcriptions.create(
                model=AUDIO_MODEL,
                file=audio_file,
                response_format="text"
            )
            return transcription
    except Exception as e:
        return f"Error during transcription: {str(e)}"

In [ ]:
# Process the uploaded audio file, transcribe it, and generate meeting minutes

def process_upload(audio_file, progress=gr.Progress()):
    progress(0.1, desc="Starting process...")

    if audio_file is None:
        return "Please upload an audio file."

    try:
        # Check file format
        if not str(audio_file).lower().endswith('.mp3'):
            return "Please upload an MP3 file."

        # Get transcription
        transcription = transcribe_audio(audio_file)
        if transcription.startswith("Error"):
            return transcription

        # Generate minutes
        minutes = generate_minutes(transcription, model, tokenizer)
        progress(1.0, desc="Process complete!")
        return minutes

    except Exception as e:
        return f"Error processing file: {str(e)}"

In [ ]:
# Create Gradio interface

interface = gr.Interface(
    fn=process_upload,
    inputs=gr.Audio(type="filepath", label="Upload MP3 File", format="mp3"),
    outputs=gr.Markdown(label="Meeting Minutes", min_height=60),
    title="Meeting Minutes Generator",
    description="Upload an MP3 recording of your meeting to get AI-generated meeting minutes. This process may take a few minutes.",
    flagging_mode="never"
)

In [ ]:
# Launch Gradio interface

interface.launch()